In [1]:
import numpy as np
import scipy as sp
%matplotlib inline
import matplotlib.pyplot as plt
from collections import defaultdict

import time
import numpy as np
from lsh import clsh, jlsh, generateSamples, findNeighborsBrute, recall

# open docs file and read its lines
with open("train.dat", "r") as fh:
    lines = fh.readlines()





train_labels=  [l.split()[0] for l in lines]


train_documents=  [l.split()[1:] for l in lines]

with open("test.dat", "r") as fh:
    test_lines = fh.readlines()

    
test_documents =  [l.split()[1:] for l in test_lines]
    
  
#print train_labels[0:10]
#print train_documents[0]


def filterLen(docs, minlen):
    r""" filter out terms that are too short. 
    docs is a list of lists, each inner list is a document represented as a list of words
    minlen is the minimum length of the word to keep
    """
    return [ [t for t in d if len(t) >= minlen ] for d in docs ]


docs1 = filterLen(train_documents, 3)





In [2]:
from collections import Counter
from scipy.sparse import csr_matrix
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        #print cnt
        keys = list(k for k,_ in cnt.most_common())
        #print keys
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat,idx


def build_test_matrix(docs,idx):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    
    
    nnz = 0
    test_docs = []
    for d in docs:
        copy_d=[]
     
        for w in d:
            if w in idx:
                copy_d.append(w)
        
        nnz += len(set(copy_d))
        test_docs.append(copy_d)
        
        
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in test_docs:
        cnt = Counter(d)
        #print cnt
        keys = list(k for k,_ in cnt.most_common())
        #print keys
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)))

In [3]:
mat,word_dict = build_matrix(train_documents)



test_mat =build_test_matrix(test_documents,word_dict)
csr_info(mat)
csr_info(test_mat)


 [nrows 25000, ncols 283110, nnz 3784021]
 [nrows 25000, ncols 283110, nnz 3626768]


In [4]:
# scale matrix and normalize its rows
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat
mat2 = csr_idf(mat, copy=True)
mat3 = csr_l2normalize(mat2, copy=True)

test_mat2 = csr_idf(test_mat, copy=True)
test_mat3 = csr_l2normalize(test_mat2, copy=True)




print("mat1:", mat[0,:12].todense(), "\n")
print("mat2:", mat2[0,:12].todense(), "\n")
print("mat3:", mat3[0,:12].todense())

('mat1:', matrix([[  2.,  11.,   1.,   1.,   1.,   1.,   6.,   1.,   1.,   1.,   5.,
           1.]]), '\n')
('mat2:', matrix([[ 7.17120221,  0.44262714,  0.76125473,  0.38748767,  4.86394091,
          6.13764706,  0.71729757,  2.35979057,  2.54593135,  5.57275421,
          3.76915902,  1.90059267]]), '\n')
('mat3:', matrix([[ 0.08526985,  0.0052631 ,  0.00905177,  0.00460746,  0.05783515,
          0.07298027,  0.00852909,  0.02805931,  0.03027264,  0.06626336,
          0.04481754,  0.02259918]]))


In [ ]:
from scipy.sparse.linalg import norm
i = 2  # one row
j = 4  # another row
# compare cosine similarity of rows from mat2 vs. mat3
dp2 = mat2[i].dot(mat2[j].T).todense().item()  # the dot-product between the sparse vectors in mat2
print "dot-product in mat2: ", dp2
print "norms in mat2: ", norm(mat2[i]), norm(mat2[j])
print "********** cosine in mat2: ", dp2 / ( norm(mat2[i]) * norm(mat2[j]) )
dp3 = mat3[i].dot(mat3[j].T).todense().item()  # the dot-product between the sparse vectors in mat3
print "dot-product in mat3: ", dp3
print "norms in mat3: ", norm(mat3[i]), norm(mat3[j])
print " ********** cosine in mat3: ", dp3 / ( norm(mat3[i]) * norm(mat3[j]) )
print mat3[2]
print mat2[2]
print mat[2]

In [21]:
import time
import numpy as np
from lsh import clsh, jlsh, generateSamples, findNeighborsBrute, recall
from scipy.sparse.linalg import norm


def cosineSimilarity(vector1, vector2):
    distance = 0
    return vector1.dot(vector2.T).todense().item() / ( norm(vector1) * norm(vector2) )


import operator 

def getKey(item):
    return item[1]

def getNeighbors(trainingSet, testInstance, k):
    distances = []
    distances=testInstance.dot(trainingSet.T)
    sims = list(zip(distances.indices, distances.data))
    sims.sort(key=lambda x: x[1], reverse=True)
    
    poscount=0
    negcount=0
    neighbors = [s[0] for s in sims[:k] if s[1] > 0 ]
    
    for x in range(k):
       
        if(train_labels[neighbors[x]]=='-1'):
            negcount+=1  
        else:
            poscount+=1
    
    label='+1'
    if(negcount>poscount):
        label='-1'

    return label




In [ ]:

import os

filename = 'output.dat'

highscore = open(filename,'w')


num_rows, num_cols = test_mat3.shape

for i in range(num_rows):
    n=getNeighbors(mat3,test_mat3[i],5)
    print n
    highscore.write(n+ '\n')


highscore.close()  
    

+1
+1
+1
+1
+1
+1
-1
+1
-1
-1
-1
+1
-1
-1
+1
+1
+1
-1
+1
-1
+1
-1
-1
-1
+1
+1
+1
+1
+1
+1
-1
+1
+1
+1
-1
+1
+1
+1
+1
-1
-1
+1
-1
+1
-1
+1
-1
-1
-1
-1
-1
-1
-1
+1
+1
-1
-1
+1
+1
+1
+1
+1
+1
-1
-1
-1
+1
+1
+1
+1
+1
-1
-1
+1
+1
-1
-1
+1
-1
+1
+1
+1
+1
+1
+1
-1
+1
+1
+1
+1
+1
-1
+1
+1
+1
-1
+1
-1
-1
+1
+1
-1
-1
-1
-1
-1
+1
-1
+1
+1
-1
-1
-1
-1
-1
-1
-1
-1
-1
+1
+1
+1
+1
-1
+1
-1
+1
+1
-1
-1
+1
-1
-1
-1
+1
+1
+1
-1
-1
-1
+1
-1
+1
+1
+1
+1
+1
+1
-1
+1
+1
+1
+1
+1
+1
+1
-1
+1
+1
-1
+1
+1
+1
+1
-1
+1
-1
-1
-1
-1
-1
-1
-1
+1
-1
-1
+1
-1
-1
-1
-1
+1
+1
+1
-1
+1
-1
-1
+1
+1
-1
-1
+1
+1
-1
+1
+1
+1
-1
+1
+1
+1
+1
-1
+1
-1
+1
-1
+1
-1
+1
+1
+1
-1
+1
+1
-1
+1
+1
+1
+1
+1
+1
-1
+1
+1
-1
-1
+1
+1
+1
+1
+1
-1
-1
+1
-1
+1
+1
+1
+1
+1
-1
-1
-1
-1
+1
+1
-1
+1
+1
+1
+1
+1
-1
-1
-1
+1
-1
+1
-1
+1
+1
-1
+1
-1
-1
-1
+1
+1
-1
+1
-1
+1
+1
+1
+1
+1
-1
+1
+1
+1
-1
-1
+1
+1
+1
-1
+1
+1
-1
-1
-1
+1
+1
-1
+1
-1
+1
-1
+1
+1
+1
+1
+1
+1
+1
+1
-1
-1
-1
+1
-1
+1
-1
+1
+1
-1
+1
-1
-1
+1
-1
+1
+1
-1
-1
+1
+1
-1
+1
+1
-1
-

-1
+1
+1
+1
+1
-1
+1
+1
-1
+1
-1
+1
+1
+1
+1
+1
+1
+1
-1
+1
-1
+1
-1
-1
-1
+1
+1
+1
-1
+1
-1
+1
-1
+1
+1
-1
-1
-1
+1
+1
-1
+1
+1
+1
+1
+1
-1
+1
-1
+1
+1
+1
+1
-1
+1
-1
+1
-1
-1
+1
+1
-1
+1
+1
+1
+1
-1
+1
+1
-1
+1
+1
+1
-1
+1
-1
+1
+1
+1
+1
+1
-1
+1
+1
+1
-1
-1
+1
+1
-1
-1
+1
-1
+1
-1
+1
-1
+1
+1
+1
+1
+1
+1
-1
-1
+1
-1
+1
-1
+1
-1
-1
+1
+1
+1
-1
+1
+1
+1
-1
+1
+1
-1
+1
+1
-1
-1
+1
+1
-1
+1
-1
-1
-1
-1
-1
-1
+1
-1
+1
-1
-1
-1
-1
+1
+1
+1
-1
+1
-1
+1
-1
-1
+1
-1
+1
+1
+1
+1
-1
+1
-1
-1
+1
+1
-1
-1
+1
-1
+1
+1
+1
-1
+1
-1
+1
-1
+1
+1
+1
+1
+1
-1
+1
-1
+1
+1
-1
+1
+1
-1
+1
+1
+1
-1
-1
+1
-1
+1
+1
-1
-1
+1
+1
+1
-1
-1
-1
+1
+1
-1
-1
-1
+1
-1
+1
+1
-1
-1
-1
+1
+1
+1
-1
-1
-1
+1
-1
-1
+1
-1
+1
+1
+1
+1
-1
-1
+1
+1
+1
+1
-1
-1
-1
-1
+1
+1
+1
-1
+1
+1
+1
+1
-1
+1
+1
+1
-1
-1
-1
+1
+1
+1
+1
+1
+1
-1
+1
-1
+1
+1
-1
+1
+1
+1
-1
+1
+1
+1
+1
-1
-1
-1
+1
-1
-1
+1
+1
-1
-1
+1
+1
+1
-1
-1
+1
+1
+1
+1
-1
-1
-1
+1
+1
+1
-1
-1
+1
-1
+1
+1
+1
-1
+1
-1
+1
-1
+1
+1
-1
+1
+1
+1
-1
+1
+1
+1
-1
+1
-1
+1
-1
+1
-

+1
+1
-1
-1
-1
-1
+1
+1
-1
+1
+1
-1
+1
+1
-1
-1
+1
+1
-1
+1
+1
+1
+1
+1
-1
+1
-1
-1
+1
+1
+1
-1
-1
+1
+1
-1
-1
+1
+1
+1
-1
-1
+1
+1
+1
+1
+1
+1
+1
+1
+1
-1
+1
+1
+1
+1
-1
-1
+1
+1
+1
-1
+1
+1
-1
+1
+1
-1
+1
-1
-1
-1
+1
-1
+1
+1
-1
+1
-1
+1
-1
-1
-1
+1
+1
+1
+1
-1
+1
+1
+1
+1
-1
+1
+1
+1
+1
-1
-1
-1
-1
-1
+1
-1
+1
+1
-1
-1
-1
+1
-1
-1
+1
+1
-1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
-1
+1
+1
-1
+1
-1
+1
+1
+1
-1
+1
-1
-1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
-1
+1
+1
-1
+1
+1
+1
+1
-1
+1
+1
-1
-1
-1
+1
+1
-1
-1
+1
+1
+1
+1
+1
+1
-1
-1
+1
+1
+1
-1
-1
+1
-1
+1
+1
+1
+1
-1
-1
+1
+1
+1
+1
+1
-1
-1
+1
+1
+1
+1
-1
+1
-1
-1
-1
+1
-1
+1
+1
+1
+1
-1
-1
+1
+1
-1
+1
+1
-1
+1
+1
-1
+1
+1
-1
+1
-1
-1
+1
+1
-1
-1
+1
+1
-1
-1
-1
+1
-1
-1
+1
+1
-1
+1
-1
-1
+1
-1
+1
+1
-1
+1
+1
-1
+1
+1
-1
+1
+1
-1
-1
-1
+1
+1
-1
+1
-1
-1
-1
+1
+1
-1
+1
-1
+1
-1
+1
-1
+1
+1
+1
-1
-1
+1
+1
-1
+1
-1
+1
-1
+1
+1
-1
-1
+1
-1
-1
-1
-1
-1
+1
+1
-1
+1
+1
-1
+1
+1
-1
+1
+1
-1
+1
+1
+1
+1
+1
+1
+1
-1
-1
-1
-1
+1
+1
+1
-1
-1
-1
+1
-1
+1
-1
+

-1
+1
+1
+1
+1
-1
+1
-1
-1
+1
+1
+1
+1
+1
+1
-1
+1
-1
+1
+1
-1
-1
+1
-1
+1
+1
+1
+1
+1
-1
+1
+1
+1
+1
-1
+1
-1
-1
+1
+1
-1
-1
-1
+1
+1
-1
+1
+1
-1
-1
+1
-1
-1
+1
-1
-1
-1
-1
-1
+1
-1
+1
+1
-1
+1
-1
-1
+1
-1
+1
+1
+1
+1
-1
+1
+1
+1
-1
-1
-1
-1
-1
-1
+1
+1
+1
+1
-1
-1
-1
-1
-1
-1
-1
+1
+1
+1
+1
+1
+1
-1
-1
+1
-1
-1
+1
-1
-1
+1
+1
-1
-1
+1
-1
+1
-1
+1
-1
+1
+1
-1
+1
-1
+1
+1
+1
+1
+1
-1
-1
-1
+1
-1
+1
+1
+1
-1
+1
+1
+1
-1
+1
-1
-1
+1
-1
+1
+1
+1
-1
+1
-1
+1
-1
-1
+1
-1
-1
+1
-1
+1
+1
+1
-1
-1
+1
+1
-1
-1
+1
-1
+1
-1
+1
-1
-1
-1
-1
+1
-1
-1
+1
+1
-1
+1
+1
+1
-1
-1
-1
+1
-1
+1
-1
+1
+1
-1
+1
-1
+1
+1
+1
+1
+1
-1
+1
+1
+1
+1
-1
-1
-1
-1
-1
+1
+1
-1
+1
+1
-1
-1
+1
+1
+1
+1
-1
+1
-1
+1
-1
+1
+1
-1
-1
-1
+1
+1
-1
+1
+1
+1
-1
-1
+1
+1
+1
-1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
-1
+1
-1
+1
-1
-1
-1
+1
-1
+1
-1
-1
+1
+1
-1
-1
+1
-1
+1
+1
+1
-1
+1
+1
-1
+1
-1
+1
+1
-1
+1
+1
+1
+1
-1
-1
+1
-1
+1
+1
-1
+1
+1
-1
+1
+1
-1
+1
-1
+1
-1
+1
+1
+1
-1
+1
-1
+1
+1
+1
+1
+1
-1
-1
-1
-1
+1
+1
+1
+1
+1
+1
+1
-1
+1
+1
-

+1
+1
-1
-1
+1
-1
-1
+1
+1
+1
-1
-1
+1
-1
+1
+1
-1
-1
-1
+1
+1
+1
+1
+1
-1
-1
-1
+1
+1
-1
+1
+1
+1
-1
-1
-1
-1
-1
+1
-1
-1
-1
-1
-1
-1
+1
+1
-1
-1
+1
-1
-1
+1
+1
-1
+1
-1
+1
+1
-1
+1
+1
+1
-1
+1
+1
+1
+1
-1
+1
+1
-1
+1
-1
+1
-1
+1
-1
+1
+1
-1
+1
+1
+1
-1
-1
-1
-1
+1
-1
+1
-1
+1
+1
+1
+1
-1
-1
-1
+1
-1
-1
-1
+1
+1
+1
+1
+1
+1
+1
-1
-1
-1
+1
+1
+1
-1
+1
+1
-1
+1
+1
+1
+1
-1
+1
+1
-1
-1
+1
+1
+1
+1
+1
+1
-1
-1
-1
+1
+1
+1
+1
+1
+1
-1
+1
-1
+1
-1
+1
-1
+1
+1
-1
-1
+1
-1
-1
+1
+1
+1
+1
+1
-1
+1
+1
-1
+1
-1
+1
-1
+1
+1
-1
+1
-1
-1
-1
+1
+1
+1
-1
-1
-1
-1
+1
+1
+1
+1
+1
+1
-1
+1
-1
+1
+1
-1
+1
+1
+1
+1
-1
-1
+1
-1
-1
-1
-1
-1
+1
-1
+1
+1
+1
-1
-1
+1
-1
+1
-1
+1
+1
-1
-1
+1
-1
+1
+1
-1
-1
+1
+1
-1
-1
+1
+1
-1
+1
+1
-1
+1
+1
-1
-1
+1
+1
+1
-1
+1
+1
+1
-1
-1
+1
+1
+1
+1
+1
+1
+1
-1
-1
-1
+1
-1
-1
+1
-1
+1
+1
-1
+1
+1
+1
-1
-1
-1
-1
-1
-1
+1
-1
-1
-1
+1
-1
-1
+1
+1
+1
+1
+1
+1
+1
+1
-1
+1
-1
-1
+1
+1
+1
-1
-1
-1
+1
+1
+1
-1
+1
+1
+1
-1
-1
-1
-1
+1
+1
-1
-1
+1
+1
-1
-1
+1
+1
+1
+1
-1
+1
+1
+1
+1
+

In [ ]:

def l2Ap(trainingSet, epsilon):
    
    
    for
    
    
    distances = []
    distances=testInstance.dot(trainingSet.T)
    sims = list(zip(distances.indices, distances.data))
    sims.sort(key=lambda x: x[1], reverse=True)
    
    poscount=0
    negcount=0
    neighbors = [s[0] for s in sims[:k] if s[1] > 0 ]
    
    for x in range(k):
       
        if(train_labels[neighbors[x]]=='-1'):
            negcount+=1  
        else:
            poscount+=1
    
    label='+1'
    if(negcount>poscount):
        label='-1'

    return label


